In [1]:
textgrid_dir = "/Users/joregan/Playing/hsi_ctmedit/textgrid"
wav_dir = "/Users/joregan/Playing/hsi/audio/"

In [2]:
from pydub import AudioSegment

In [3]:
from pathlib import Path

textgrid_path = Path(textgrid_dir)
wav_path = Path(wav_dir)

In [4]:
from praatio import textgrid

In [21]:
import re

def noise_event(text):
    m = re.match("^\[([^\]]+)\]$", text)
    return (m is not None)

In [ ]:
for tgfile in textgrid_path.glob("*.[Tt]ext[Gg]rid"):
    wavname = f"{tgfile.stem}.wav"
    audio = AudioSegment.from_file(wav_path / wavname)
    resampled = audio.set_frame_rate(16000)
    # del(audio)
    tg = textgrid.openTextgrid(tgfile, False)



In [26]:
def get_tier_entries(textgrid, tiername="utterances"):
    entries = []

    tier = textgrid.getTier(tiername)
    for entry in tier.entries:
        text = entry.label
        if text.strip() == "":
            continue
        if not noise_event(text.strip()):
            entries.append({
                "start": entry.start,
                "end": entry.end,
                "text": entry.label
            })
    return entries

In [28]:
def run_espeak(text):
    phon = !echo "{text}" | espeak -v en-us --ipa -q
    return " ".join(phon)

In [29]:
def espeakify_entries(entries):
    for entry in entries:
        entry["espeak"] = run_espeak(entry["text"])

In [30]:
ent = get_tier_entries(tg)

In [31]:
espeakify_entries(ent)

In [32]:
ent

[{'start': 0.4819935034938646,
  'end': 4.070102841824428,
  'text': 'Okay, so we can do the landlord stuff.',
  'espeak': ' oʊkˈeɪ  sˌoʊ wiː kæn dˈuː ðə lˈændlɔːɹd stˈʌf '},
 {'start': 5.8815966619896445,
  'end': 8.660968384458382,
  'text': 'So yeah, thanks for renting me out this place.',
  'espeak': ' sˌoʊ jˈɛh  θˈæŋks fɔːɹ ɹˈɛntɪŋ mˌiː ˈaʊt ðɪs plˈeɪs '},
 {'start': 22.42615370788416,
  'end': 25.564154039703702,
  'text': "It's a bit small for me, I want a bigger apartment.",
  'espeak': ' ɪts ɐ bˈɪt smˈɔːl fɔːɹ mˌiː  aɪ wˈɑːnt ɐ bˈɪɡɚɹ ɐpˈɑːɹtmənt '},
 {'start': 30.993257898679108,
  'end': 32.16505713264647,
  'text': 'Yes of course.',
  'espeak': ' jˈɛs ʌv kˈoːɹs '},
 {'start': 46.180879624851336,
  'end': 47.07745114822835,
  'text': 'No.',
  'espeak': ' nˈoʊ '},
 {'start': 52.1457772986572,
  'end': 53.53120956350062,
  'text': 'Ah I think it was like that before.',
  'espeak': ' ˈɑː aɪ θˈɪŋk ɪt wʌz lˈaɪk ðæt bɪfˈoːɹ '},
 {'start': 62.23376541427553,
  'end': 63.41259093575